# Dataclass Builder Demo

Interactive visual dataclass construction with sPyTial and CnD-core.

In [1]:
from dataclasses import dataclass
import spytial
from spytial import attribute

This demo shows how to create interactive visual builders for Python dataclasses using sPyTial. 

**How it works:**
1. Build your data structure visually in the CnD interface
2. Click **"Export JSON"** button - this downloads a file
3. Save the file to the directory shown by the widget
4. Access the result with `widget.value` or `widget.refresh()`

Features:
- Visual construction of dataclass instances
- File-based export (works in all environments)
- Support for spatial annotations
- Works with any dataclass structure

In [2]:
@dataclass
@attribute(field='value')
class TreeNode:
    value: int = 0
    left: 'TreeNode' = None
    right: 'TreeNode' = None

In [3]:
tree_widget = spytial.dataclass_builder(TreeNode)
tree_widget

In [ ]:
# After exporting and saving the file, check the result:
tree = tree_widget.value
print(f"Built tree: {tree}")
if tree:
    print(f"Root value: {tree.value}")
    print(f"Has left child: {tree.left is not None}")
    print(f"Has right child: {tree.right is not None}")
else:
    print("\nNo data yet. Steps:")
    print("1. Build your tree in the widget above")
    print("2. Click 'Export JSON' button")
    print(f"3. Save file to: {tree_widget._export_dir}")
    print("4. Re-run this cell")

In [ ]:
# Check if the widget value was updated
print("Checking widget value after Export...")
print(f"tree_widget.value = {tree_widget.value}")
print()
print("If this is None, the message handler didn't receive the data.")
print("Expected to see Python output like:")
print("  📥 Received data (length: XX chars)")
print("  📊 Data structure keys: [...]")
print("  ✅ Built: TreeNode(...)")
print()
print("If you don't see that output above, the JavaScript → Python communication failed.")

In [ ]:
# Debug: Check what data structure is being passed to CnD
from spytial.provider_system import CnDDataInstanceBuilder
import json

empty_tree = TreeNode()
builder = CnDDataInstanceBuilder()
init_data = builder.build_instance(empty_tree)

print("Initial data passed to CnD interface:")
print(json.dumps(init_data, indent=2)[:500] + "...")
print(f"\nNumber of atoms: {len(init_data.get('atoms', []))}")
print(f"Number of relations: {len(init_data.get('relations', []))}")
print(f"Number of types: {len(init_data.get('types', []))}")